# 2022-02-07 • Big-N-to-1 simulation

In [1]:
# Pkg.resolve()

In [4]:
include("nb_init.jl")

[ Info: using Revise
[ Info: import Distributions
[ Info: import PyPlot
[ Info: import DataFrames, PrettyTables
[ Info: import MyToolbox
[ Info: using VoltageToMap


In [29]:
using Parameters, ComponentArrays

@alias CArray = ComponentArray;

In [6]:
save(fname) = savefig(fname, subdir="methods");

## Parameters

In [8]:
sim_duration = 10 * seconds
Δt = 0.1 * ms;

### Input spikers

In [9]:
N_unconn = 100
N_exc    = 5200
N_inh    = N_exc ÷ 4

1300

In [10]:
N_conn = N_inh + N_exc

6500

In [11]:
N = N_conn + N_unconn

6600

In [12]:
input_spike_rate = LogNormal_with_mean(4Hz, √0.6)  # See the previous notebook

LogNormal{Float64}(μ=1.0862943611198905, σ=0.7745966692414834)

### Synapses

Reversal potential at excitatory and inhibitory synapses,  
as in the report [`2021-11-11__synaptic_conductance_ratio.pdf`](https://github.com/tfiers/phd-thesis/blob/main/reports/2021-11-11__synaptic_conductance_ratio.pdf):

In [13]:
v_exc =   0 * mV
v_inh = -65 * mV;

Exponential decay time constant of synaptic conductance `g`, $τ_{s}$ (`s` for "synaptic"):

In [14]:
τs =   7 * ms;

### Izhikevich neuron

Membrane potential `v` and adaptation variable `u` at `t = 0`:

In [15]:
v0    = -80 * mV
u0    =   0 * pA;

Parameters for a cortical regular spiking neuron:

In [16]:
@with_kw struct IzhikevichParams
    C      = 100 * pF
    k      = 0.7 * (nS/mV)
    b      = -2 * nS
    v_r    = -60 * mV
    v_t    = -40 * mV
    v_peak =  35 * mV
    c      = -50 * mV
    a      = 0.03 / ms
    d      = 100 * pA
end

cortical_RS = IzhikevichParams();

## Neuron IDs

In [17]:
neuron_ids = CArray(exc = 1:N_exc, inh = 1:N_inh, unconn = 1:N_unconn)

ComponentVector{Int64}(exc = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  5191, 5192, 5193, 5194, 5195, 5196, 5197, 5198, 5199, 5200], inh = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300], unconn = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100])

In [18]:
only(getaxes(neuron_ids))

Axis(exc = 1:5200, inh = 5201:6500, unconn = 6501:6600)

In [19]:
showsome(labels(neuron_ids))

6600-element Vector{String}:
    1: "exc[1]"
    2: "exc[2]"
    ⋮
 2838: "exc[2838]"
    ⋮
 4479: "exc[4479]"
    ⋮
 6599: "unconn[99]"
 6600: "unconn[100]"


i.e. a neuron's **global** ID = its index into the [ComponentVector](https://github.com/jonniedie/ComponentArrays.jl) "`neuron_ids`".

## Inputs

In [21]:
λ = rand(input_spike_rate, N)  # sample firing rates, one for every input neuron
β = 1 ./ λ                     # alternative Exp parametrisation: scale (= 1 / rate)
ISI_distributions = Exponential.(β);
#   This uses julia's broadcasting `.` syntax: make an `Expontential` distribution for every value in the β vector

In [23]:
# Create v_syn vector: for each neuron, the reversal potential at its downstream synapses.
vs = CArray(E=fill(v_exc, N_exc), I=fill(v_inh, N_inh))

ComponentVector{Float64}(E = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], I = [-0.065, -0.065, -0.065, -0.065, -0.065, -0.065, -0.065, -0.065, -0.065, -0.065  …  -0.065, -0.065, -0.065, -0.065, -0.065, -0.065, -0.065, -0.065, -0.065, -0.065])

## Sim

Proof of concept of spike generation using a priority queue.

In [26]:
using DataStructures

first_spiketimes = rand.(ISI_distributions)

pq = PriorityQueue{Int, Float64}()
for (input_neuron, t) in enumerate(first_spiketimes)
    enqueue!(pq, input_neuron => t)
end

t = 0s
while t < sim_duration
    input_neuron, t = dequeue_pair!(pq)  # earliest spike
    new_ISI = rand(ISI_distributions[input_neuron])
    enqueue!(pq, input_neuron => t + new_ISI)
end

Superfast.

In [31]:
using OrdinaryDiffEq

In [32]:
function f(D, vars, params, t)
    @unpack C, k, b, v_r, v_t, v_peak, c, a, d = params
    @unpack v, u = vars
    D.v = (k * (v - v_r) * (v - v_t) - u) / C
    D.u = a * (b * (v - v_r) - u)
    D.g = -g / τ_syn
    return nothing
end

x0 = ComponentArray{Float64}(v = v0, u = u0)  # note eltype cast to float
prob = ODEProblem(f, x0, float(sim_duration), cortical_RS)
# integrator = init(prob, Tsit5(); Δt, adaptive=true)

ODEProblem with uType ComponentVector{Float64, Vector{Float64}, Tuple{Axis{(v = 1, u = 2)}}} and tType Float64. In-place: true
timespan: (0.0, 10.0)
u0: ComponentVector{Float64}(v = -0.08, u = 0.0)

In [ ]:
t = 0ms:0.1ms:sim_duration
v = t -> sol(t).v / mV
plot(t, v.(t));